In [8]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import time as time
from datetime import datetime
from collections import defaultdict
import numpy as np
from sklearn.preprocessing import LabelEncoder
import networkx as nx
import pickle

In [9]:
letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
player_data = []

for letter in letters:
    url = f"https://www.footballdb.com/players/current.html?letter={letter}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        player_rows = soup.find_all('div', class_='tr')
        for row in player_rows:
            player_name_tag = row.find('div', class_='td').find('a', href=True)
            if player_name_tag:
                player_name = player_name_tag.get_text(strip=True)
            position_tag = row.find_all('div', class_='td')[1]
            player_position = position_tag.get_text(strip=True) if position_tag else 'N/A'
            player_data.append([player_name, player_position])

    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
df_names = pd.DataFrame(player_data, columns=["Player Name", "Position"])
print(df_names)
names_list = df_names['Player Name'].tolist()
    

             Player Name Position
0     Abanikanda, Israel       RB
1        Abdullah, Ameer       RB
2        Abdullah, Yasir       LB
3       Abernathy, Micah       DB
4         Abraham, Micah       DB
...                  ...      ...
2563      Zeitler, Kevin       OG
2564      Zierer, Kilian       OT
2565         Zinter, Zak       OG
2566      Zuerlein, Greg        K
2567      Zylstra, Shane       TE

[2568 rows x 2 columns]


In [ ]:
# Cell takes about ten minutes to run
nfl_teams = [
    "arizona-cardinals", "atlanta-falcons", "baltimore-ravens", "buffalo-bills",
    "carolina-panthers", "chicago-bears", "cincinnati-bengals", "cleveland-browns",
    "dallas-cowboys", "denver-broncos", "detroit-lions", "green-bay-packers",
    "houston-texans", "indianapolis-colts", "jacksonville-jaguars", "kansas-city-chiefs",
    "los-angeles-chargers", "los-angeles-rams", "miami-dolphins", "minnesota-vikings",
    "new-england-patriots", "new-orleans-saints", "new-york-giants", "new-york-jets",
    "philadelphia-eagles", "pittsburgh-steelers", "san-francisco-49ers", "seattle-seahawks",
    "tampa-bay-buccaneers", "tennessee-titans", "washington-commanders"
]

counter = 0
player_data = []
D = 2005
print(D)
for yearo in range(D, 2024):
    year = 4026 - yearo
    time.sleep(10) # Add some sleep time to not overload request limit
    for team in nfl_teams: 
        url = f"https://www.footballdb.com/teams/nfl/{team}/roster/{year}"
        if team == "washington-commanders" and year >= 1936 and year < 2020:
            url = f"https://www.footballdb.com/teams/nfl/washington-redskins/roster/{year}"
        elif team == "washington-commanders" and (year == 2021 or year == 2020):
            url = f"https://www.footballdb.com/teams/nfl/washington-football-team/roster/{year}"
        elif team == "los-angeles-rams" and year >= 1995 and year <= 2015:
            url = f"https://www.footballdb.com/teams/nfl/st-louis-rams/roster/{year}"
        elif team == "los-angeles-chargers" and year <= 2016:
            url = f"https://www.footballdb.com/teams/nfl/san-diego-chargers/roster/{year}"
        elif team == "tennessee-titans" and year <= 1998:
            url = f"https://www.footballdb.com/teams/nfl/tennessee-oilers/roster/{year}"    
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            print(f"Got data for the {team} in year {year}")
            soup = BeautifulSoup(response.text, 'html.parser')
            roster_table = soup.find('div', class_='divtable divtable-striped divtable-mobile')
            player_rows = roster_table.find_all('div', class_='tr')
            
            for row in player_rows:
                name_tag = row.find('div', class_='td w20 m80 rostercell_name')
                player_name_tag = name_tag.find('a', href=True) if name_tag else None
                player_name = player_name_tag.get_text(strip=True) if player_name_tag else 'N/A'
                position_tag = row.find('div', class_='td w10 rostercell_pos hidden-xs')
                player_position = position_tag.get_text(strip=True) if position_tag else 'N/A'
                player_data.append([team, year, player_name, position_tag])

        else:
            print(f"FAILED to retrieve the page for team {team} in year {year}. Status code: {response.status_code}")
df = pd.DataFrame(player_data, columns=["Team", "Year", "Player Name", "Player Position"])
print(f"Successfully made player df of shape: ", df.shape)
df.to_csv(f"team_composition_{D}_to_2023.csv", index=False, encoding="utf-8")

2005
Got data for the arizona-cardinals in year 2021
Got data for the atlanta-falcons in year 2021
Got data for the baltimore-ravens in year 2021
Got data for the buffalo-bills in year 2021
Got data for the carolina-panthers in year 2021
Got data for the chicago-bears in year 2021
Got data for the cincinnati-bengals in year 2021
Got data for the cleveland-browns in year 2021
Got data for the dallas-cowboys in year 2021
Got data for the denver-broncos in year 2021
Got data for the detroit-lions in year 2021
Got data for the green-bay-packers in year 2021
Got data for the houston-texans in year 2021
Got data for the indianapolis-colts in year 2021
Got data for the jacksonville-jaguars in year 2021
Got data for the kansas-city-chiefs in year 2021
Got data for the los-angeles-chargers in year 2021
Got data for the los-angeles-rams in year 2021
Got data for the miami-dolphins in year 2021
Got data for the minnesota-vikings in year 2021
Got data for the new-england-patriots in year 2021
Got 

In [14]:
print(df)
nfl_team_compositions = df
print(nfl_team_compositions)
nfl_team_compositions['Team'] = nfl_team_compositions['Team'].apply(lambda x: x.replace(" ", "").replace("-", "").upper())
print(nfl_team_compositions)

                        Team  Year     Player Name Player Position
0          arizona-cardinals  2021   Robert Alford            [DB]
1          arizona-cardinals  2021      Zach Allen            [DE]
2          arizona-cardinals  2021     Budda Baker            [DB]
3          arizona-cardinals  2021     Chris Banjo            [DB]
4          arizona-cardinals  2021  Kelvin Beachum            [OT]
...                      ...   ...             ...             ...
39067  washington-commanders  2003      Kevin Ware            [TE]
39068  washington-commanders  2003      Ron Warner            [DE]
39069  washington-commanders  2003   Brandon Winey            [OT]
39070  washington-commanders  2003    Renaldo Wynn            [DE]
39071  washington-commanders  2003   Peppi Zellner            [DE]

[39072 rows x 4 columns]
                        Team  Year     Player Name Player Position
0          arizona-cardinals  2021   Robert Alford            [DB]
1          arizona-cardinals  2021  

In [27]:
def get_nfl_year(date_str):
    game_date = datetime.strptime(date_str, "%Y-%m-%d")
    if game_date.month in [1, 2]:  # For January and February, it's the previous NFL season
        return game_date.year - 1
    else:
        return game_date.year

In [ ]:
df = pd.read_csv('/Users/bradleymoon/downloads/cs224w/GridironGraph/nfl-game-info.csv')
df['Date'] = df['Date'].apply(get_nfl_year)

def determine_winner_loser(row):
    if row['Home Team'] == "washington-redskins" and row['Date'] >= 1936 and row['Date'] < 2020:
        row['Home Team'] = "washington-commanders"
    elif row['Home Team'] == "washington-football-team" and (row['Date'] == 2021 or row['Date'] == 2020):
        row['Home Team'] = "washington-commanders"
    elif row['Away Team'] == "washington-redskins" and row['Date'] >= 1936 and row['Date'] < 2020:
        row['Away Team'] = "washington-commanders"
    elif row['Away Team'] == "washington-football-team" and (row['Date'] == 2021 or row['Date'] == 2020):
        row['Away Team'] = "washington-commanders"
    
    if row['Home Team'] == "st-louis-rams" and row['Date'] >= 1995 and row['Date'] <= 2015:
        row['Home Team'] = "los-angeles-rams"
    elif row['Away Team'] == "st-louis-rams" and row['Date'] >= 1995 and row['Date'] <= 2015:
        row['Away Team'] = "los-angeles-rams"
    
    if row['Away Team'] == "san-diego-chargers" and row['Date'] <= 2016:
        row['Away Team'] = "los-angeles-chargers"
    elif row['Home Team'] == "san-diego-chargers" and row['Date'] <= 2016:
        row['Home Team'] = "los-angeles-chargers"

    if row['Home Team'] == "tennessee-oilers" and row['Date'] <= 1998:
        row['Home Team'] = "tennessee-titans"
    elif row['Away Team'] == "tennessee-oilers" and row['Date'] <= 1998:
        row['Away Team'] = "tennessee-titans"

    # Now compare the scores to determine the winner and loser
    if row['Home Score'] > row['Away Score']:
        return pd.Series([row['Home Team'], row['Away Team']])
    elif row['Home Score'] < row['Away Score']:
        return pd.Series([row['Away Team'], row['Home Team']])
    else:
        return pd.Series([None, None])

# Apply the function to determine winners and losers, and handle exceptions
df[['Winning Team', 'Losing Team']] = df.apply(determine_winner_loser, axis=1)
df['Winning Team'] = df['Winning Team'].apply(lambda x: x.replace(" ", "").replace("-", "").upper() if pd.notnull(x) else x)
df['Losing Team'] = df['Losing Team'].apply(lambda x: x.replace(" ", "").replace("-", "").upper() if pd.notnull(x) else x)
df.to_csv('nfl-game-info-updated.csv', index=False)
print(df.head())

   Date              Home Team           Away Team  Home Score  Away Score  \
0  2024    Pittsburgh Steelers     New York Giants          26          18   
1  2024    San Francisco 49ers      Dallas Cowboys          30          24   
2  2024         Denver Broncos   Carolina Panthers          28          14   
3  2024      Las Vegas Raiders  Kansas City Chiefs          20          27   
4  2024  Washington Commanders       Chicago Bears          18          15   

  Overtime? Playoff Game? Neutral Venue?  Home Odds Open  Home Odds Min  ...  \
0       NaN           NaN            NaN            1.54           1.29  ...   
1       NaN           NaN            NaN            1.42           1.29  ...   
2       NaN           NaN            NaN            1.50           1.12  ...   
3       NaN           NaN            NaN            4.50           4.25  ...   
4       NaN           NaN            NaN            1.80           1.80  ...   

   Total Score Over Min  Total Score Over Max  Tot

In [32]:
D = 2005
nfl_game_info = pd.read_csv('nfl-game-info-updated.csv')
nfl_team_compositions = pd.read_csv(f'team_composition_{D}_to_2023.csv')
nfl_team_compositions['Team'] = nfl_team_compositions['Team'].apply(lambda x: x.replace(" ", "").replace("-", "").upper())
team_to_players = defaultdict(lambda: defaultdict(list))

for _, row in nfl_team_compositions.iterrows():
    team = row['Team']
    year = row['Year']
    player = row['Player Name']
    team_to_players[team][year].append(player)
adj_matrix = defaultdict(lambda: defaultdict(int))
for _, game in nfl_game_info.iterrows():
    winner = game['Winning Team']
    loser = game['Losing Team']
    nfl_year = game['Date']
    
    if winner in team_to_players and nfl_year in team_to_players[winner]:
        winning_players = team_to_players[winner][nfl_year]
    else:
        continue

    if loser in team_to_players and nfl_year in team_to_players[loser]:
        losing_players = team_to_players[loser][nfl_year]
    else:
        continue
    for winner_player in winning_players:
        for loser_player in losing_players:
            adj_matrix[winner_player][loser_player] += 1

adj_matrix_df = pd.DataFrame.from_dict({player: dict(edges) for player, edges in adj_matrix.items()}).fillna(0)
print(adj_matrix_df)


                  Cam Akers  Landen Akers  Brian Allen  Tremayne Anchrum  \
Hakeem Adeniji          1.0           1.0          5.0               1.0   
Brandon Allen           1.0           1.0          5.0               1.0   
Ricardo Allen           1.0           1.0          5.0               1.0   
Eli Apple               1.0           1.0          5.0               1.0   
Chidobe Awuzie          2.0           1.0          6.0               2.0   
...                     ...           ...          ...               ...   
Troy Bienemann          0.0           0.0          0.0               0.0   
David Holloway          0.0           0.0          0.0               0.0   
Bhawoh Jue              0.0           0.0          0.0               0.0   
Mel Mitchell            0.0           0.0          0.0               0.0   
Marcellus Rivers        0.0           0.0          0.0               0.0   

                  Chatarius Atwell  Odell Beckham Jr.  Kendall Blanton  \
Hakeem Adenij

In [ ]:
D = 2005
nfl_game_info = pd.read_csv('nfl-game-info-updated.csv')
nfl_team_compositions = pd.read_csv(f'team_composition_{D}_to_2023.csv')
nfl_team_compositions['Team'] = nfl_team_compositions['Team'].apply(lambda x: x.replace(" ", "").replace("-", "").upper())

team_to_players = defaultdict(lambda: defaultdict(list))

for _, row in nfl_team_compositions.iterrows():
    team = row['Team']
    year = row['Year']
    player = row['Player Name']
    team_to_players[team][year].append(player)

all_players = set()
for team in team_to_players.values():
    for players in team.values():
        all_players.update(players)

print(len(all_players))
player_encoder = {player: idx for idx, player in enumerate(sorted(all_players))} # USE IN FUTURE PARTS
adj_matrix = defaultdict(lambda: defaultdict(int))

for _, game in nfl_game_info.iterrows():
    winner = game['Winning Team']
    loser = game['Losing Team']
    nfl_year = game['Date']
    
    if winner in team_to_players and nfl_year in team_to_players[winner]:
        winning_players = team_to_players[winner][nfl_year]
    else:
        continue

    if loser in team_to_players and nfl_year in team_to_players[loser]:
        losing_players = team_to_players[loser][nfl_year]
    else:
        continue

    for winner_player in winning_players:
        winner_idx = player_encoder[winner_player]
        for loser_player in losing_players:
            loser_idx = player_encoder[loser_player]
            adj_matrix[winner_idx][loser_idx] += 1

adj_matrix_df = pd.DataFrame.from_dict({player: dict(edges) for player, edges in adj_matrix.items()}).fillna(0)
print(adj_matrix_df)

9213
      1253  5646  1038  8678  1486  6835  5237  468   790   8345  ...  6157  \
3494   1.0   1.0   5.0   1.0   1.0   5.0   1.0   2.0   1.0   1.0  ...   0.0   
871    1.0   1.0   5.0   1.0   1.0   5.0   1.0   2.0   1.0   1.0  ...   0.0   
7334   1.0   1.0   5.0   1.0   1.0   4.0   2.0   1.0   1.0   1.0  ...   0.0   
3038   1.0   1.0   5.0   1.0   1.0   3.0   2.0   1.0   1.0   1.0  ...   0.0   
1505   2.0   1.0   6.0   2.0   1.0   4.0   1.0   2.0   1.0   2.0  ...   0.0   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
8757   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
2359   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
732    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
6313   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
5942   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   

      7341  3438  8602  2864  3176  7287  18  

In [36]:
adj_matrix_np = adj_matrix_df.to_numpy()
total_elements = adj_matrix_np.size
print(total_elements)
zero_elements = np.sum(adj_matrix_np == 0)
print(zero_elements)
sparsity = zero_elements / total_elements

print(f"Sparsity with naive representation: {sparsity:.4f}")

import scipy.sparse as sp

adj_matrix_sparse = sp.csr_matrix(adj_matrix_df.to_numpy())
sparsity_sparse = 1 - (adj_matrix_sparse.nnz / adj_matrix_sparse.size)
print(adj_matrix_sparse)
print(f"Sparsity with sparse matrix: {sparsity_sparse:.4f}")

61583146
51472838
Sparsity with naive representation: 0.8358
  (0, 0)	1.0
  (0, 1)	1.0
  (0, 2)	5.0
  (0, 3)	1.0
  (0, 4)	1.0
  (0, 5)	5.0
  (0, 6)	1.0
  (0, 7)	2.0
  (0, 8)	1.0
  (0, 9)	1.0
  (0, 10)	1.0
  (0, 11)	1.0
  (0, 12)	1.0
  (0, 13)	1.0
  (0, 14)	1.0
  (0, 15)	1.0
  (0, 16)	1.0
  (0, 17)	1.0
  (0, 18)	1.0
  (0, 19)	1.0
  (0, 20)	1.0
  (0, 21)	1.0
  (0, 22)	1.0
  (0, 23)	1.0
  (0, 24)	1.0
  :	:
  (7857, 6438)	1.0
  (7857, 6530)	1.0
  (7857, 6709)	1.0
  (7857, 6775)	1.0
  (7857, 6776)	1.0
  (7857, 6778)	1.0
  (7857, 6779)	1.0
  (7857, 6781)	1.0
  (7857, 6794)	1.0
  (7857, 6996)	1.0
  (7857, 6997)	1.0
  (7857, 6998)	1.0
  (7857, 7000)	1.0
  (7857, 7002)	1.0
  (7857, 7004)	1.0
  (7857, 7006)	1.0
  (7857, 7155)	1.0
  (7857, 7156)	1.0
  (7857, 7157)	1.0
  (7857, 7158)	1.0
  (7857, 7159)	1.0
  (7857, 7160)	1.0
  (7857, 7161)	1.0
  (7857, 7162)	1.0
  (7857, 7163)	1.0
Sparsity with sparse matrix: 0.0000


In [41]:
G = nx.DiGraph()
for winner_idx, losers in adj_matrix.items():
    for loser_idx, count in losers.items():
        G.add_edge(winner_idx, loser_idx, weight=count)
        
print(f"Graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")

Graph has 7858 nodes and 10110308 edges.
